# 协程

# 迭代器
- 可迭代（Interable）：直接作用于for循环的变量
- 迭代器（Interator）：不但可以直接作用于for循环，还可以被next调用
    - 迭代器一定可迭代，但可迭代并不一定是迭代器
- list是典型的可迭代对象，但不是迭代器
- 通过isinstance判断
- iterable和iterator可以转换
    - 通过iteration函数

In [2]:
# 可迭代
l = [i for i in range(10)]

for idx in l:
    print(idx,end='')
    
# range是个迭代器
for i in range(5):
    print(i)

01234567890
1
2
3
4


In [8]:
# isinstance判断某个变量是否是一个的子类

# 判断是否可迭代
import collections
l = [i for i in range(5)]
print(isinstance(l, collections.Iterable))

# 是否是迭代器
print(isinstance(l, collections.Iterator))

True
False


In [12]:
# iter函数，迭代器和可迭代相互转换
s = "i love ma xiaoxiao"
print(isinstance(s, collections.Iterable))
print(isinstance(s, collections.Iterator))

iter_s = iter(s)
print(isinstance(iter_s, collections.Iterable))
print(isinstance(iter_s, collections.Iterator))

True
False
True
True


# 生成器
- generator：一遍循环一边计算下一个元素的机制或算法
- 需要满足三个条件：
    - 每次调用都产生出for循环需要的下一个元素
    - 如果达到最后一个数，爆出StopIteration异常
    - 可以被next函数调用
    
- 如何生存一个生成器
    - 直接使用
    - 如果函数中包含yield，则这个函数就叫生成器
    - next调用函数，遇到yield返回

In [13]:
# 直接使用生成器

L = [x*x for x in range(5)]  # 放在中括号中是列表生成器
g = (x*x for x in range(5))  # 放在小括号中就是生成器
print(type(L))
print(type(g))

<class 'list'>
<class 'generator'>


In [21]:
# 案例
def odd():
    print("Step 1 ")
    print("Step 2")
    print("Step 3")
    return None

odd()
print("=================")
# 生成器案例
def odd():
    print("Step 1 ")
    yield 1
    print("Step 2")
    yield 2
    print("Step 3")
    yield 3
    return None
# odd()是调用生成器
g = odd()
one = next(g)
two = next(g)
print(one)
print(two)

Step 1 
Step 2
Step 3
Step 1 
Step 2
1
2


In [22]:
# for循环调用生成器
def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        print(b)
        a,b = b, a+b
        n += 1
    return "Done"
fib(5)

1
1
2
3
5


'Done'

In [24]:
# 裴波那契数列生成器写法
def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        yield b
        a,b = b, a+b
        n += 1
    # 爆出异常值的是return的返回值   
    return "Done"

g = fib(5)
for i in range(6):
    rst = next(g)
    print(rst)

1
1
2
3
5


StopIteration: Done

# 协程
- 实现协程的比较好的包有asyncio,tornado,gevent
- 定义：协程是为非抢占式多任务产生子程序的计算机程序组件，协程允许不同入口点在不同位置暂停或开始执行程序
- 从技术角度讲，协程就是一个你可以暂停执行的函数，或者可以把它理解成生成器也行
- 协程的实现：
    - yield返回
    - send调用
- 协程的四个状态
    - inspect.getgeneratorstate(...)函数确定，该函数会返回下列字符串中的一个：
        - GEN_CREATED:等待开始执行
        - GEN_RUNNING:解释器正在执行
        - GEN_SUSPEND:在yield表达式出暂停
        - GEN_CLOSED：执行结束
    - next预激（prime）
- 协程终止：
    - 协程中未处理的异常会向上冒泡，传给next函数或send方法的调用方（即触发协程的对象）
    - 终止协程的一种方式：发送某个哨符值，让协程退出。内置的None和Ellipsis等常量经常用作哨符值
- yield from（相当于是一个中间人）
    - 调用协程为了得到返回值，协程必须正常终止
    - 生成器正常终止会发出StopIteration异常，异常对象的vlaue属性保存返回值
    - yield from从内部捕获StopIteration异常
    - 委派生成器
        - 包含yield from表达式的生成器函数
        - 委派生成器在yield from表达式处暂停，调用方可以直接把数据发给子生成器
        - 子生成器再把产出的值发给调用方
        - 子生成器在最后，解释器会抛出StopIteration，并且把返回值附加到异常对象上

In [29]:
# 协程代码案例：
def coroutine():
    print("i am starting")
    yield 1
    print("i have received")

sc = coroutine()
print('程序执行')
next(sc)  # 预激
print('卡一下')
next(sc)



程序执行
i am starting
卡一下
i have received


StopIteration: 

In [32]:
def simpole_coroutine(a):
    print("i am starting")
    b = yield a
    print("copy :",a,b)
    
    c = yield a + b
    print("copy :",a, b, c)
    
sc = simpole_coroutine(8)
a = next(sc)
print(a)
b = sc.send(9)
print(b)
c = sc.send(10)
print(c)

i am starting
8
copy : 8 9
17
copy : 8 9 10


StopIteration: 

In [50]:
# yield from案例
def gen():
    for i in "AB":
        yield i
print(list(gen()))

#对于gen来讲，list直接从gen中得到数据
# 对于gen_new来讲，list先从yield from中索要数据，然后yield from再从gen_new索要数据，最后层层返回
def gen_new():
    yield from "AB"
print(list(gen_new()))
print(type(gen()))

['A', 'B']
['A', 'B']
<class 'generator'>


In [45]:
# 委派生成器
import collections

ResClass = collections.namedtuple("Res", 'count average')

# 子生成器
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total / count
    return ResClass(count, average)

# 委派生成器
def grouper(storages, key):
        while True:
            storages[key] = yield from averager()
            
#  客户端代码
def client():
    process_data = {
        'boy_2':[27, 23.3, 23.6, 49.1, 34.5, 26.5, 45.8],
        'boy_1':[1.34, 1.5, 1.53, 1.48, 1.39, 1.36]
    }
    storages = {}
    for k, v in process_data.items():
        # 获得协程
        coroutine = grouper(storages, k)
        
        # 预激协程
        next(coroutine)
        
        # 发送数据到协程
        for dt in v:
            coroutine.send(dt)
        
        # 终止协程
        coroutine.send(None)
    print(storages)
    
# run
client()

{'boy_2': Res(count=7, average=32.82857142857143), 'boy_1': Res(count=6, average=1.4333333333333333)}


In [46]:
process_data = {
        'boy_2':[27, 23.3, 23.6, 49.1, 34.5, 26.5, 45.8],
        'boy_1':[1.34, 1.5, 1.53, 1.48, 1.39, 1.36]
    }

for k, v in process_data.items():
    print(k)
    print(v)

boy_2
[27, 23.3, 23.6, 49.1, 34.5, 26.5, 45.8]
boy_1
[1.34, 1.5, 1.53, 1.48, 1.39, 1.36]


# asyncio
- 内置对异步io的支持
- asyncio本身是一个消息循环
- 步骤：
    - 创建消息循环
    - 把协程导入
    - 关闭

In [59]:
import threading
# 引入异步io包
import asyncio

# 使用协程
@asyncio.coroutine  # 这里是协程的固定写法，需要用该修饰器

def hello():
    print("hello world! {}".format(threading.currentThread()))
    print("Start .....{}".format(threading.currentThread()))
    yield from asyncio.sleep(1)
    print("Done.....{}".format(threading.currentThread()))
    print("Hello again! {}".format(threading.currentThread))

# 启动消息循环
loop = asyncio.get_event_loop()
# 定义任务
tasks = [hello(), hello()]
# asyncio使用wait等待task执行完毕
loop.run_until_complete(asyncio.wait(tasks))
#　关闭消息循环
loop.close()
# 错误是Spyder不支持造成的

RuntimeError: This event loop is already running

hello world! <_MainThread(MainThread, started 3424)>
Start .....<_MainThread(MainThread, started 3424)>
hello world! <_MainThread(MainThread, started 3424)>
Start .....<_MainThread(MainThread, started 3424)>
Done.....<_MainThread(MainThread, started 3424)>
Hello again! <function current_thread at 0x0000016DCD7497B8>
Done.....<_MainThread(MainThread, started 3424)>
Hello again! <function current_thread at 0x0000016DCD7497B8>


In [63]:
import asyncio

@asyncio.coroutine
def wget(host):
    print('wget %s...'  % host)
    # 异步请求网址
    connect = asyncio.open_connection(host, 80)
    # 注意yield from的写法
    reader, writer = yield from connect
    header = 'GET / HTTP/1.0\r\nHost: %s\r\n\r\n' % host
    writer.write(header.encode('utf-8'))
    yield from writer.drain()
    while True:
        line = yield from reader.readline()
        # http协议的换行使用\r\n
        if line == b'\r\n':
            break
        print('%s header > %s' % (host, line.decode('utf-8').rstrip()))
    # Ignore the body, close the socket
    writer.close()
    
loop = asyncio.get_event_loop()
tasks = [wget(host) for host in ['www.sina.com.cn', 'www.sohu.com', 'www.baidu.com']]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

RuntimeError: This event loop is already running

wget www.sina.com.cn...
wget www.sohu.com...
wget www.baidu.com...
www.sina.com.cn header > HTTP/1.1 302 Moved Temporarily
www.sina.com.cn header > Server: nginx
www.sina.com.cn header > Date: Wed, 14 Aug 2019 08:57:02 GMT
www.sina.com.cn header > Content-Type: text/html
www.sina.com.cn header > Content-Length: 154
www.sina.com.cn header > Connection: close
www.sina.com.cn header > Location: https://www.sina.com.cn/
www.sina.com.cn header > X-Via-CDN: f=edge,s=ctc.chengdu.ha2ts4.27.nb.sinaedge.com,c=171.88.100.242;
www.sina.com.cn header > X-Via-Edge: 1565773022349f26458ab3850d3de484671e2
www.sohu.com header > HTTP/1.1 200 OK
www.sohu.com header > Content-Type: text/html;charset=UTF-8
www.sohu.com header > Connection: close
www.sohu.com header > Server: nginx
www.sohu.com header > Date: Wed, 14 Aug 2019 08:56:07 GMT
www.sohu.com header > Access-Control-Allow-Credentials: true
www.sohu.com header > Vary: Origin,Access-Control-Request-Method,Access-Control-Request-Headers
www.sohu.com he

# async and await
- 为了更好地异步，替换asyncio让协程更加简单
    - 替换方式：用async替换@asynico.coroutine
    - await替换yield from
    

In [1]:
import threading
import asyncio

async def hello():
    print("hello world! {}".format(threading.currentThread()))
    print("Start...{}".format(threading.currentThread()))
    await asynico.sleep(2)
    print("Done .....{}".foramt(threading.currentThread()))
    print("hello again.....{}".format(threading.currentThread()))
    
loop = asyncio.get_event_loop()
tasks = [hello(), hello()]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

RuntimeError: This event loop is already running

hello world! <_MainThread(MainThread, started 7160)>
Start...<_MainThread(MainThread, started 7160)>
hello world! <_MainThread(MainThread, started 7160)>
Start...<_MainThread(MainThread, started 7160)>


# aiohttp
- asyncio实现单线程的并发io，在客户端用处不大
- 在服务器端可以asyncio+coroutine配合，因为http是io操作
- asynico是基于nsyncio是实现的http框架


In [3]:
# aiohttp案例

import asyncio

from aiohttp import web

async def index(request):
    await asyncio.sleep(0.4)
    return web.Response(body=b'<h1>Index</h1>')
    
async def hello(request):
    await asyncio.sleep(0.5)
    text = '<h1>hello, %s!</h1>' % request.match_info['name']
    return web.Response(body=text.encode('utf-8'))
async def init(loop):
    app = web.Application(loop=loop)
    app.router.add_route('GET', '/', index)
    app.router.add_route('GET', '/hello/{name}', hello)
    srv = await loop.create_server(app.make_handler(), '127.0.01', 8000)
    print("server started at http://127.0.0.1:8000...")
    return srv

loop1 = asyncio.get_event_loop()
loop1.run_until_complete(init(loop))
loop1.run_forever()

RuntimeError: This event loop is already running

e:\Users\acer\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: loop argument is deprecated
  app.launch_new_instance()
e:\Users\acer\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Application.make_handler(...) is deprecated, use AppRunner API instead


# concurrent.futures
- 类似其它语言的线程池的概念
- 利用multiprocessing实现真正的并行计算
- 核心原理：以子进程的形式，并行运行多个python解释器
    从而令python程序可以多核CPU来提升执行速度
    由于子进程与主解释器相分离，所以他们的全局解释器锁也是相互独立的
    每个子进程都能够完整的使用一个CPU内核
- - concurrent.futures.Executor
    - ThreadPoolExecutor
    - ProcessPoolExecutor
    - 执行的时候需要自行选择
- submit（fn, args, keargs）
    - fn:异步执行的函数
    - args.kwargs参数

In [14]:
# 关于concurrent案例

from concurrent.futures import ThreadPoolExecutor
import time

def yourage(age):
    time.sleep(2)
    print('your age is {}'.format(age))
    return age
def yourname(msg):
    time.sleep(2)
    print('your name is {}'.format(msg))
    return msg

# 创建一个线程池
pool = ThreadPoolExecutor(max_workers=2)

# 王线程池中加入2个task
f1 = pool.submit(yourage, '22')
f2 = pool.submit(yourname, 'taogege')

print(f1.done())
time.sleep(3)
print(f2.done())

print(f1.result())
print(f2.result())
f1.

False
your age is yearoldyour name is taogege

True
yearold
taogege


# current中map函数
- map函数回忆
        map()是 Python 内置的高阶函数，它接收一个函数 f 和一个 list，并通过把函数 f 依次作用在 list 的每个元素上，得到一个新的 list 并返回。例如，对于list [1, 2, 3, 4, 5, 6, 7, 8, 9]，f(x)=x*x
- map（fn,\*iterables, timeout=None）
    - 跟map函数类似
    - 函数需要异步执行
    - timeout：超时时间
    - map和submit使用一个就行
# Future


In [15]:
# map案例

import time,re
import os, datetime
from concurrent import futures

data = ['1','2']

def wait_on(argument):
    print(argument)
    time.sleep(2)
    return "ok"

ex = futures.ThreadPoolExecutor(max_workers=2)
for i in ex.map(wait_on, data):
    print(i)

1
2
ok
ok
